In [ ]:
from openpyxl import *

In [49]:
class _excel:
    def __init__(self, name):
        self.name = name
        filename = self.name + ".xlsx"
        workbook = Workbook()
        workbook.save(filename=filename)
    def get_sheet(self):
        workbook = load_workbook(filename=self.name + ".xlsx")
        return workbook.sheetnames
    def add_col(self, col_name):
        workbook = load_workbook(filename=self.name + ".xlsx")
        col_name = workbook.create_sheet(col_name, 0)
        workbook.save(filename=self.name + ".xlsx")


In [50]:
sheet1 = _excel("Test")

In [51]:
sheet1.get_sheet()

['Sheet']

In [52]:
sheet2 = _excel("Test2")

In [55]:
sheet2.get_sheet()

['Sheet', 'IP Addressses']

In [54]:
sheet2.add_col("IP Addressses")